In [1]:
import pandas as pd
import re
import numpy as np

In [34]:
df = pd.read_csv('../data/articles_supp.csv')

/home/rdora/miniconda3/envs/femec/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [135]:
article = pd.read_csv('../data/article.csv')

In [136]:
article.head()

,Author-Handle,Short-Id,Item-Type,Item-Id
0,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:fem:femwpa:2014.46
1,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:ecl:stabus:3097
2,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:arx:papers:1405.6400
3,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:ags:feemcl:172702
4,repec:per:2004-04-21:leonce_ndikumana,pnd1,paper,repec:uma:periwp:wp5


# Add number of authors

In [36]:
nauthors = df.groupby('handle').size().rename('n_authors').reset_index()

In [37]:
df = pd.merge(df, nauthors, how='left')

# Articles

In [53]:
df['year'] = df.year.astype(str)

In [58]:
def remove_zero(x):
    if x[-2:] == '.0':
        return x[:-2]
    else:
        return x

In [59]:
df['year'] = df['year'].apply(remove_zero)

In [81]:
def remove_hyphen(x):
    if "-" in x:
        s = x.split('-')
        if len(s[0]) == 4:
            return s[0]
        else:
            return x
    else:
        return x

In [82]:
df['year'] = df['year'].apply(remove_hyphen)

In [101]:
def remove_romans(x):
    r = re.compile(r'\d{4}(?= *\()')
    m = r.match(x)
    if m:
        return m.group()
    else:
        return x

In [103]:
df['year'] = df['year'].apply(remove_romans)

In [114]:
def remove_nondigit(x):
    r = re.compile(r'\d{4}(?!=\d)')
    m = r.match(x)
    if m:
        return m.group(0)
    else:
        return x

In [116]:
df['year'] = df['year'].apply(remove_nondigit)

In [127]:
def apply_nan(x):
    r = re.compile(r'^\d{4}$')
    if r.match(x):
        return x
    else:
        return np.nan

In [128]:
df['year'] = df['year'].apply(apply_nan)

## Replace weird stuff

By hand

In [139]:
df['year'] = df['year'].replace({'2917': '2017'})

In [142]:
df['year'] = df['year'].replace({'2952': '2010'})

In [147]:
df['year'] = df['year'].replace({'1000': '1999'})

In [150]:
df = df[df.year.notna()]

In [156]:
df = df.drop_duplicates(subset='handle')

# Add gender

In [162]:
article.head()

,Author-Handle,Short-Id,Item-Type,Item-Id
0,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:fem:femwpa:2014.46
1,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:ecl:stabus:3097
2,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:arx:papers:1405.6400
3,repec:per:2017-11-30:stephen_nei,pne333,paper,repec:ags:feemcl:172702
4,repec:per:2004-04-21:leonce_ndikumana,pnd1,paper,repec:uma:periwp:wp5


In [163]:
df_us = df[['handle', 'year', 'n_authors']].rename(columns={'handle': 'Item-Id',
                                                            'n_authors': 'n_authors_base'})

In [171]:
df_us['Item-Id'] = df_us['Item-Id'].str.lower().str.strip()

In [173]:
article_2 = pd.merge(article,
                     df_us,
                     how="left")

In [183]:
article_2 = article_2[article_2['Item-Type']=='article']

## Add year with parsing

In [186]:
article_2_good = article_2[article_2.year.notna()]

In [187]:
article_2_bad = article_2[article_2.year.isna()]

In [188]:
article_2_bad = article_2_bad.drop('year', axis=1)

In [199]:
article_2_bad['year'] = article_2_bad['Item-Id'].str.extract(r'((?<=y:)\d{4}(?=:))')

In [200]:
article_3 = pd.concat([article_2_good, article_2_bad])

## N authors

In [203]:
n_authors = article_3.groupby('Item-Id').size().rename('n_authors').reset_index()

In [204]:
article_3 = pd.merge(article_3, n_authors, how='left')

In [207]:
article_3 = article_3.drop('n_authors_base', axis=1)

# Gender

In [212]:
authors = pd.read_csv('../data/processed/people.csv')

In [215]:
article_3 = pd.merge(article_3, authors, how='left')

In [219]:
article_3.to_csv('../data/econometry/base_1_todos.csv', index=False)